    Copyright 2021 Google LLC

    Licensed under the Apache License, Version 2.0 (the "License");
    you may not use this file except in compliance with the License.
    You may obtain a copy of the License at

        https://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing, software
    distributed under the License is distributed on an "AS IS" BASIS,
    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    See the License for the specific language governing permissions and
    limitations under the License.


# Dialogflow CX Bot Language Translation (Agent Notebook)

Contains functions that are re-used in the Main Notebook

1. Functions for Querying CX Agent for Intents, Entities, Flows, Pages and Route Groups
2. Functions for Updating CX Agent with translated text from Sheets

[Public Doc Link: Python Client for CX](https://cloud.google.com/dialogflow/cx/docs/reference/library/python)

## QUICK Doc REF: CX Agent Component Types & Hierarchy

### Types

In scope: `Type`, **bold** for required attributes, *italics* for not in scope

- `Intent`
    - **name**:str
    - **display_name**:str
    - training_phrases:`[ ] of Intent.TrainingPhrase`
        - `TrainingPhrase`
            - id:str (output only)
            - **parts**:`[ ] of Intent.TrainingPhrase.Part`
                - `Part`
                    - **text**:str
                    - parameter_id:str
            - **repeat_count**:int (def: 1)
    - parameters:`[ ] of Intent.Parameter`
        - `Parameter`
            - **id**:str
            - **entity_type**:str (entity_type uri)
            - is_list:bool
            - redact:bool
    - *priority:int*
    - *is_fallback:bool*
    - *labels:`[ ] of Intent.LabelsEntry`*
        - *`LabelsEntry` (proto.message.Message)*
    - *description:str*
---
- `EntityType`
    - **name**:str
    - **display_name**:str
    - **kind**:`EntityType.Kind`
        - `Kind(value)`
            - KIND_LIST = 2
            - KIND_MAP = 1
            - KIND_REGEXP = 3
            - KIND_UNSPECIFIED = 0
    - *auto_expansion_mode: `EntityType.AutoExpansionMode`*
        - *`AutoExpansionMode(value)`*
            - *AUTO_EXPANSION_MODE_DEFAULT = 1*
            - *AUTO_EXPANSION_MODE_UNSPECIFIED = 0*
    - entities:`[ ] of EntityType.Entity`
        - `Entity`
            - **value**:str
            - **synonyms**:[ ] of str
    - *excluded_phrases:`[ ] of EntityType.ExcludedPhrase`*
        - *`ExcludedPhrase`*
            - **value**:str
    - *enable_fuzzy_extraction:bool*
    - *redact:bool*
---
- `Flow`
    - **name**:str
    - **display_name**:str
    - *description:str*
    - transition_routes:`[ ] of `<mark>TransitionRoute</mark>
    - event_handlers:`[ ]  of `<mark>EventHandler</mark>
    - transition_route_groups:[ ] of str
    - *nlu_settings: NluSettings*
---
- `TransitionRouteGroups` (aka Route Groups)
    - **name**:str
    - **display_name**:str
    - transition_routes:[] of <mark>TransitionRoute</mark>
---
- `Pages`
    - **name**:str
    - **display_name**:str
    - entry_fulfillment:<mark>Fulfillment</mark>
    - form:<mark>Form</mark>
    - transition_route_groups:[ ] of str
    - transition_routes:[ ] of <mark>TransitionRoute</mark>
    - event_handlers:[ ]  of <mark>EventHandler</mark>
---
- <mark>Form</mark>
    - parameters:[ ] of `Form.Parameter`
        - `Parameter`
            - **display_name**:str
            - *required:bool*
            - *entity_type:str*
            - *is_list:bool*
            - fill_behavior:`Form.Parameter.FillBehavior`
                - `FillBehavior`
                    - initial_prompt_fulfillment:<mark>Fulfillment</mark>
                    - reprompt_event_handlers:`[ ] of `<mark>EventHandler</mark>
            - *default_value:google.protobuf.struct_pb2.Value*
            - *redact:bool*
            
- <mark>TransitionRoute</mark>
    - name:str (output only)
    - intent:str
    - condition:str
    - trigger_fulfillment: <mark>Fulfillment</mark>
    - *target_page:str*
    - *target_flow:str*

- <mark>EventHandler<mark>
    - name:str (output only)
    - **event**:str
    - trigger_fulfillment: <mark>Fulfillment</mark>
    - *target_page:str*
    - *target_flow:str*

- <mark>Fulfillment</mark>
    - messages:`[ ] of `<mark>ResponseMessage</mark>
    - *webhook:str*
    - *return_partial_responses:bool*
    - *tag:str*
    - *set_parameter_actions:[ ] of Fulfillment.SetParameterAction*
        - *SetParameterAction*
            - *parameter:str*
            - *value:google.protobuf.struct_pb2.Value*
    - *conditional_cases:[ ] of Fulfillment.ConditionalCases*

- <mark>ResponseMessage</mark>
    - text:`ResponseMessage.Text`
        - `Text`
            - **text**:[ ] of str
            - *allow_playback_interruption:bool*
    - *payload:google.protobuf.struct_pb2.Struct*
    - *conversation_success:ResposneMessage.ConversationSuccess*
    - output_audio_text:`ResponseMessage.OutputAudioText`
        - `OutputAudioText`
            - *text:str*
            - ssml:str
            - *allow_playback_interruption:bool*
    - *live_agent_handoff:ResponseMessage.LiveAgentHandoff*
    - *end_interaction:ResponseMessage.EndInteraction (output only)*
    - *play_audio:ResponseMessage.PlayAudio*
    - *mixed_audio:ResponseMessage.MixedAudio*

---

Out of scope but listed for completeness:
- Agents
- Deployments
- Environments
- Experiments
- SecuritySettingsService
- SessionEntityTypes
- Sessions
- TestCases
- Versions
- Webhooks

# Agent

## ipynb env:

In [ ]:
# !python3 -V
# !python3 -m pip list | wc -l
# !python3 -m pip list | grep google

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Import & Init

## CX Libs & Vars

In [ ]:
# CX Libs
import argparse
import uuid
#V3 Beta 1
from google.cloud.dialogflowcx_v3beta1.services.agents import AgentsClient
from google.cloud.dialogflowcx_v3beta1.services.sessions import SessionsClient
from google.cloud.dialogflowcx_v3beta1.services.intents import IntentsClient
from google.protobuf import field_mask_pb2

import google.cloud.dialogflowcx_v3beta1.services as cx_services
import google.cloud.dialogflowcx_v3beta1.types as cx_types

import bs4

# Functions: CX - Query Agent Configuration

## Functions

In [ ]:
### According to https://cloud.google.com/dialogflow/quotas#table
### Design-time requests are set to 60 requests per minute (or 1 request per second)

from timeit import default_timer as timer
import time

_last_cx_request_time = timer()

### spacing out API calls to at most once per second since last invocation
def delay_next_cx_request():
    global _last_cx_request_time
    gap = timer() - _last_cx_request_time
    if __DEBUG: print(f'delay_next_cx_request(): time gap between last API call is {gap}s')
    if (gap < 1):
        sleep_time = round(1 - gap, 3)
        if __INFO: print(f'\tdelay_next_cx_request(): Sleeping {sleep_time}s as duration from last API call is {gap}s')
        time.sleep(sleep_time)
    _last_cx_request_time = timer()


## Intents

In [ ]:
#######################
def list_intents():
    
    intents_client = IntentsClient(credentials=get_cx_credentials(), 
                                   client_options=get_cx_client_options())
    intent_request = cx_types.ListIntentsRequest()
    intent_request.parent = CX_Agent_Link
    delay_next_cx_request()
    return intents_client.list_intents(intent_request) #caller to handle exceptions

###########################
def get_intent(name, lang):
    
    intents_client = IntentsClient(credentials=get_cx_credentials(), 
                                   client_options=get_cx_client_options())
   
    intent_request = cx_types.GetIntentRequest()
    intent_request.name = name
    intent_request.language_code = lang
    delay_next_cx_request()
    return intents_client.get_intent(intent_request) #caller to handle exceptions
    

In [ ]:
#################################
def get_training_phrases(intent):
    training_phrases_list = []
    
    for training_phrase in intent.training_phrases:
        training_phrase_list = []
        parts_list = []
        for part in training_phrase.parts:
            if part.parameter_id == "":
                parts_list.append(part.text)
            else:
                parts_list.append(f'<b data-parameter_id="{part.parameter_id}">{part.text}</b>')
        
        training_phrase_list.append(intent.name)
        training_phrase_list.append(intent.display_name)
        training_phrase_list.append("".join(parts_list))
        training_phrases_list.append(training_phrase_list)
    
    return training_phrases_list
    
###########################
def get_parameters(intent):
    parameters_list = []
    
    for parameter in intent.parameters:
        parameter_list = []
        parameter_list.append(intent.name)
        parameter_list.append(intent.display_name)
        parameter_list.append(parameter.id)
        parameter_list.append(parameter.entity_type)
        parameter_list.append(parameter.is_list)
        parameter_list.append(parameter.redact)
        parameters_list.append(parameter_list)
            
    return parameters_list


In [ ]:
##############################
def write_intents_to_sheets():
    start_time = timer()
    print("START:\t\twrite_intents_to_sheets()\n")
    try:
        intents = list_intents()
    except Exception as e:
        raise Exception(f'write_intents_to_sheets(): Exception at list_intents().  See below:\n{e}')
    else:
        tp_row = 4
        param_row = 2
        tp_values = []
        param_values = []
        for intent in intents:
            tp_values += get_training_phrases(intent)
            param_values += get_parameters(intent)
        
        try:
            clear_sheet("Training_Phrases!A4:C")
            print(f"write_intents_to_sheets(): intent:[{intent.display_name}]: Writing {len(tp_values)} lines of Training Phrases to Sheets.")
            write_to_sheet(f'Training_Phrases!A{tp_row}', tp_values, 'RAW', 'append')
        except Exception as e:
            raise Exception(f'write_intents_to_sheets(): Exception at write_to_sheet("Training_Phrases!A{tp_row}", tp_values, "RAW").  See below:\n{e}')

        try:
            clear_sheet("Parameters!A2:F")
            print(f"write_intents_to_sheets(): intent:[{intent.display_name}]: Writing {len(param_values)} lines of Parameters to Sheets.")
            write_to_sheet(f'Parameters!A{param_row}', param_values, 'RAW', 'append')
        except Exception as e:
            raise Exception(f'write_intents_to_sheets(): Exception at write_to_sheet("Parameters!A{param_row}", param_values, "RAW").  See below:\n{e}')
                
        print(f"\nCOMPLETED:\twrite_intents_to_sheets() in {timer()-start_time}s\n")


## Entity Types

In [ ]:
########################
def list_entity_types():
    entity_client = cx_services.entity_types.EntityTypesClient(credentials=get_cx_credentials(), 
                                                               client_options=get_cx_client_options())
    entity_request = cx_types.ListEntityTypesRequest()
    entity_request.parent = CX_Agent_Link
    delay_next_cx_request()
    return entity_client.list_entity_types(entity_request) #caller to handle exceptions


########################################
def get_entity_types(entity_name, lang):
    entity_client = cx_services.entity_types.EntityTypesClient(credentials=get_cx_credentials(), 
                                                               client_options=get_cx_client_options())
    entity_request = cx_types.GetEntityTypeRequest()
    entity_request.name = entity_name
    entity_request.language_code = lang
    delay_next_cx_request()
    return entity_client.get_entity_type(entity_request) #caller to handle exceptions


In [ ]:
def write_entities_to_sheets():
    start_time = timer()
    print("START:\t\twrite_entities_to_sheets()\n")
    #Entities
    try:
        entity_types = list_entity_types()
    except Exception as e:
        raise Exception(f'write_entities_to_sheets(): Exception at list_entity_types().  See below:\n{e}')
    else:
        entity_type_names = []
        entity_type_display_names = []
        entity_type_kinds = []
        entity_type_entity_values  = []
        entity_type_entity_synonyms = []

        for entity_type in entity_types:
            entity_type_names.append(entity_type.name)
            entity_type_display_names.append(entity_type.display_name)
            entity_type_kinds.append(entity_type.kind.value)

            ent_values = []
            ent_values_synonyms = []
            for entity in entity_type.entities:
                ent_values.append(entity.value)
                ent_synonyms = []
                for synonym in entity.synonyms:
                    ent_synonyms.append(synonym)
                ent_values_synonyms.append(ent_synonyms)

            entity_type_entity_values.append(ent_values)
            entity_type_entity_synonyms.append(ent_values_synonyms)

        i = 0
        values = []
        while i < len(entity_type_names):
            j = 0
            while j < len(entity_type_entity_values[i]):
                k = 0
                while k < len(entity_type_entity_synonyms[i][j]):
                    values.append([ entity_type_names[i],entity_type_display_names[i],
                                    entity_type_kinds[i],entity_type_entity_values[i][j],
                                    entity_type_entity_synonyms[i][j][k] ])
                    k += 1
                j += 1
            i += 1

        #print(values)
        try:
            clear_sheet("Entities!A4:E")
            print(f"write_entities_to_sheets(): Writing {len(values)} lines of Entity_Types to Sheets.")
            write_to_sheet('Entities!A4', values, 'RAW', 'append')
        except Exception as e:
            raise Exception(f'write_entities_to_sheets(): Exception at write_to_sheet("Entities!A4", values, "RAW").  See below:\n{e}')
        else:
            print(f"\nCOMPLETED:\twrite_entities_to_sheets() in {timer()-start_time}s\n")

## Flows

In [ ]:
#################
def list_flows():
    flows_client = cx_services.flows.FlowsClient(credentials=get_cx_credentials(), 
                                                 client_options=get_cx_client_options())
    flows_request = cx_types.ListFlowsRequest()
    flows_request.parent = CX_Agent_Link
    delay_next_cx_request()
    return flows_client.list_flows(flows_request) #caller to handle exceptions

#############################
def get_flow(name,lang_code):
    flows_client = cx_services.flows.FlowsClient(credentials=get_cx_credentials(), 
                                                 client_options=get_cx_client_options())
    flows_request = cx_types.GetFlowRequest(name=name,language_code=lang_code)
    delay_next_cx_request()
    return flows_client.get_flow(flows_request) #caller to handle exceptions

**Captured Items:**
- name
- display_name
- transition_routes (trigger_fulfillment with messages:text/s,output_audio_text)
- event_handlers (trigger_fulfillment with text/s)

**Excluded Item:**
- transition_route_groups (only listed but no details)
    - require a separate API call to retrieve (text search to determine)
    - nested (move to a separate process)

**Every Flow has:**
- 1 name
- 1 display_name
- multiple transition_routes
    - multiple trigger_fulfillments messages
        - multiple text (also SSML - [output_audio_text])
- multiple event_handlers
    - multiple trigger_fulfillments messages
        - multiple text (also SSML - [output_audio_text])

In [ ]:
def write_flows_to_sheets():
    start_time = timer()
    print("START:\t\twrite_flows_to_sheets()\n")
    #Flows
    try:
        flows = list_flows()
    except Exception as e:
        raise Exception(f'write_flows_to_sheets(): Exception at list_flows().  See below:\n{e}')
    else:
        sheet_row = 4
        clear_sheet("Flows!A4:E")
        for flow in flows:
            sheet_values = generate_sheets_values(flow)
            if __DEBUG:
                print("Sheet Values:\n")
                for sheet in sheet_values:
                    print(f'{sheet}\n\n')
            try:
                print(f"write_flows_to_sheets(): flow:[{flow.display_name}]: Writing {len(sheet_values)} lines to Sheets.")
                write_to_sheet(f"Flows!A{sheet_row}", sheet_values, 'RAW', 'append')
            except Exception as e:
                raise Exception(f'write_flows_to_sheets(): Exception at write_to_sheet("Flows!A{sheet_row}", sheet_values, "RAW").  See below:\n{e}')
            finally:
                sheet_row += len(sheet_values)
        print(f"\nCOMPLETED:\twrite_flows_to_sheets() in {timer()-start_time}s\n")

## Pages

In [ ]:
########################
def list_pages(flow_id):
    pages_client = cx_services.pages.PagesClient(credentials=get_cx_credentials(), 
                                                 client_options=get_cx_client_options())
    pages_request = cx_types.ListPagesRequest(parent=flow_id)
    delay_next_cx_request()
    return pages_client.list_pages(pages_request)  #caller to handle exceptions
    
#################################
def get_page(page_id, lang_code):
    pages_client = cx_services.pages.PagesClient(credentials=get_cx_credentials(), 
                                                 client_options=get_cx_client_options())
    pages_request = cx_types.GetPageRequest(name=page_id,language_code=lang_code)
    delay_next_cx_request()
    return pages_client.get_page(pages_request)  #caller to handle exceptions
    

**Captured Items:**
- name
- display_name
- entry_fulfillment
- form
- transition_routes (trigger_fulfillment with messages:text/s,output_audio_text)
- event_handlers (trigger_fulfillment with text/s)

**Excluded Item:**
- transition_route_groups (only listed but no details)
    - require a separate API call to retrieve (text search to determine)
    - nested (move to a separate process)

**Every Page has:**
- 1 name
- 1 display_name
- 1 entry_fulfillment
- 1 form with 1..n parameters
- multiple transition_routes
    - multiple trigger_fulfillments messages
        - multiple text (also SSML - [output_audio_text])
- multiple event_handlers
    - multiple trigger_fulfillments messages
        - multiple text (also SSML - [output_audio_text])

In [ ]:
def write_pages_to_sheets():
    start_time = timer()
    print("START:\t\twrite_pages_to_sheets()\n")
    
    #Pages - loop through flows to get pages
    try:
        flows = list_flows()
    except Exception as e:
        raise Exception(f'write_pages_to_sheets(): Exception at list_flows().  See below:\n{e}')
    else:
        sheet_row = 4
        clear_sheet("Pages!A4:G")
        for flow in flows:
            try:
                pages = list_pages(flow.name)
            except Exception as e:
                raise Exception(f'write_pages_to_sheets(): Exception at list_pages(flow.name).  See below:\n{e}')
            else:
                for page in pages:
                    sheet_values = generate_sheets_values(page)
                    try:
                        print(f"write_pages_to_sheets(): flow:[{flow.display_name}], page:[{page.display_name}]: Writing {len(sheet_values)} lines to Sheets.")
                        write_to_sheet(f"Pages!A{sheet_row}", sheet_values, 'RAW', 'append')
                    except Exception as e:
                        raise Exception(f'write_pages_to_sheets(): Exception at write_to_sheet("Pages!A{sheet_row}", sheet_values, "RAW").  See below:\n{e}')
                    finally:
                        sheet_row += len(sheet_values)
        print(f"\nCOMPLETED:\twrite_pages_to_sheets() in {timer()-start_time}s\n")

## Route Groups

In [ ]:
###############################
def list_route_groups(flow_id):
    route_groups_client = cx_services.transition_route_groups.TransitionRouteGroupsClient(credentials=get_cx_credentials(), 
                                                                                      client_options=get_cx_client_options())
    route_groups_request = cx_types.ListTransitionRouteGroupsRequest(parent=flow_id)
    delay_next_cx_request()
    return route_groups_client.list_transition_route_groups(route_groups_request)  #caller to handle exceptions
    
###########################################
def get_route_group(route_group,lang_code):
    route_groups_client = cx_services.transition_route_groups.TransitionRouteGroupsClient(credentials=get_cx_credentials(), 
                                                                                      client_options=get_cx_client_options())

    route_groups_request = cx_types.GetTransitionRouteGroupRequest(name=route_group,language_code=lang_code)
    delay_next_cx_request()
    return route_groups_client.get_transition_route_group(route_groups_request)  #caller to handle exceptions
    

In [ ]:
def write_route_groups_to_sheets():
    start_time = timer()
    print("START:\t\twrite_route_groups_to_sheets()\n")

    # Route Groups - loop through flows to get them
    try:
        flows = list_flows()
    except Exception as e:
        raise Exception(f'write_route_groups_to_sheets(): Exception at list_flows().  See below:\n{e}')
    else:
        sheet_row = 4
        clear_sheet("Route_Groups!A4:E")
        for flow in flows:
            try:
                route_groups = list_route_groups(flow.name)
            except Exception as e:
                raise Exception(f'write_route_groups_to_sheets(): Exception at list_route_groups({flow.name}).  See below:\n{e}')
            else:
                for route_group in route_groups:
                    sheet_values = generate_sheets_values(route_group)
                    print(f"write_route_groups_to_sheets(): route_group:[{route_group.display_name}]: Writing {len(sheet_values)} lines to Sheets.")
                    try:
                        write_to_sheet(f"Route_Groups!A{sheet_row}", sheet_values, 'RAW', 'append')
                    except Exception as e:
                        raise Exception(f'write_route_groups_to_sheets(): Exception at write_to_sheet("Route_Groups!A{sheet_row}", sheet_values, "RAW").  See below:\n{e}')
                    finally:
                        sheet_row += len(sheet_values)
        
        print(f"\nCOMPLETED:\twrite_route_groups_to_sheets() in {timer()-start_time}s\n")


# Functions: CX - Update Agent

## Functions

In [ ]:
############################################
def isSSML(text):
    upper = text.upper()
    return upper.startswith('<SPEAK>')

############################################
def generate_cx_messages(df):
    text_list = []
    ssml_list = []
    for text in df[df.columns[len(df.columns)-1]]:
        #print(text)
        if isSSML(text): #ssml
            ssml_list.append(text)
        else: #text
            text_list.append(text)

    messages = []
    if text_list != []:
        text = cx_types.ResponseMessage.Text()
        text.text = text_list
        #print(text)
        message = cx_types.ResponseMessage()
        message.text = text
        messages.append(message)
    if ssml_list != []:
        output_audio_texts = []
        for ssml in ssml_list:
            output_audio_text = cx_types.ResponseMessage.OutputAudioText()
            output_audio_text.ssml = ssml
            output_audio_texts.append(output_audio_text)
        #print(output_audio_texts)
        for oat in output_audio_texts:
            message = cx_types.ResponseMessage()
            message.output_audio_text = oat
            messages.append(message)
    
    return messages

##########################################################################
def merge_messages(src_msgs, tgt_msgs):
    if __DEBUG:
        print(f'length of src_msgs:{len(src_msgs)}')
        print(f'length of tgt_msgs:{len(tgt_msgs)}')
        if len(src_msgs) > len(tgt_msgs):
            for src_msg in src_msgs:
                print(f'scr_msg.text:[{src_msg.text}]')
            for tgt_msg in tgt_msgs:
                print(f'tgt_msg.text:[{tgt_msg.text}]')
    
    if tgt_msgs != []:
        if src_msgs == []:
            src_msgs = tgt_msgs
        else: ### BIG ASSUMPTION HERE:  src and tgt msgs are same length!
            i = 0
            while i < len(src_msgs):
                if __DEBUG:
                    print(f'while loop i:{i}')
                    print(f'len(tgt_msgs[{i}].text.text):[{len(tgt_msgs[i].text.text)}]')
                if len(tgt_msgs[i].text.text) > 0:
                    if __DEBUG:
                        print(f'tgt_msgs[{i}].text:[{tgt_msgs[i].text}]')
                    src_msgs[i].text = tgt_msgs[i].text
                if len(tgt_msgs[i].output_audio_text.ssml) > 0:
                    src_msgs[i].output_audio_text = tgt_msgs[i].output_audio_text
                i += 1
    return src_msgs

## Intents

In [ ]:
###############################################
def update_intent(assembled_intent, lang_code):
    print(f"START: update_intent(assembled_intent:[{assembled_intent.display_name}],lang_code:[{lang_code}])")
    
    intents_client = IntentsClient(credentials=get_cx_credentials(), 
                                   client_options=get_cx_client_options())
    
    intent_request = cx_types.intent.UpdateIntentRequest(intent=assembled_intent,
                                                         language_code=lang_code)
    update_mask = field_mask_pb2.FieldMask(paths=['display_name','training_phrases','parameters'])
    intent_request.update_mask = update_mask
    
    delay_next_cx_request()
    start_time = timer()
    response = intents_client.update_intent(request=intent_request)
    #print(response)
    print(f"COMPLETED: update_intent(assembled_intent:[{assembled_intent.display_name}],lang_code:[{lang_code}]) in {timer()-start_time}s")

####################################
def process_training_phrase(tp_str):
    tp_parts = []
    if __DEBUG:
        print(f'process_training_phrase: tp_str:\n{tp_str}')
    tp_soup = bs4.BeautifulSoup(tp_str,'html.parser')

    for element in tp_soup:
        if isinstance(element, bs4.element.NavigableString):
            tp_parts.append(cx_types.Intent.TrainingPhrase.Part(text=element))
        elif isinstance(element, bs4.element.Tag):
            if element.name == 'b':
                tp_parts.append(cx_types.Intent.TrainingPhrase.Part(text=element.string,parameter_id=element.get('data-parameter_id','no_parameter_id_found')))
    
    return tp_parts

#################################
def assemble_intent(intent_dict):
    tps_df = intent_dict.get("training_phrases")
    params_df = intent_dict.get("parameters")
    
    intent = cx_types.Intent()
    intent.name = intent_dict.get("intent_name")
    intent.display_name = intent_dict.get("intent_display_name")
    training_phrases_list = []
    parameters_list = []
    
    ## Training Phrases
    tps = tps_df[intent_dict.get("target_lang")]
    for tp in tps:
        training_phrase = cx_types.Intent.TrainingPhrase()
        training_phrase.parts = process_training_phrase(tp)
        training_phrase.repeat_count = 1            
        training_phrases_list.append(training_phrase)
    intent.training_phrases = training_phrases_list
    
    ## Parameter Prompts
    parameter_IDs = params_df["Parameter ID"].unique().tolist()
    for parameter_id in parameter_IDs:
        parameter = cx_types.Intent.Parameter(id=parameter_id)
        p_filter  = params_df["Parameter ID"] == parameter_id
        parameter.entity_type = params_df[p_filter]["Parameter Entity Type"].values[0]
        parameter.is_list     = params_df[p_filter]["Boolean:Is_List"].values[0] == "True"
        parameter.redact      = params_df[p_filter]["Boolean:Redact"].values[0] == "True"
        parameters_list.append(parameter)
        
    intent.parameters = parameters_list
        
    return intent
    
#########################
def update_all_intents():
    start_time = timer()
    print("START:\t\tupdate_all_intents()\n")
    
    training_phrases_df = read_sheet(SheetsName.Training_Phrases.name)
    parameters_df = read_sheet(SheetsName.Parameters.name)

    target_langs = training_phrases_df.columns.tolist()
    del target_langs[0:3]
    print(f'target_langs:{target_langs}')
    
    intent_names = training_phrases_df.loc[3:,"Intent Name"].unique().tolist()
    intent_names_dict = {}
    for intent_name in intent_names:
        intent_names_dict[intent_name] = training_phrases_df[training_phrases_df["Intent Name"]==intent_name]["Intent Display Name"].values[0]

    for target_lang in target_langs:
        for intent_name in intent_names_dict.keys():
            print(f'\nintent_name:[{intent_name}]\nintent_display_name:[{intent_names_dict.get(intent_name)}], target_lang:[{target_lang}]')
            intent_dict = {}
            intent_dict["training_phrases"] = training_phrases_df.loc[training_phrases_df["Intent Name"]==intent_name,
                                                                      ["Intent Name", "Intent Display Name", target_lang]]

            intent_dict["parameters"] = parameters_df.loc[parameters_df["Intent Name"]==intent_name,
                                                          ["Intent Name","Intent Display Name",
                                                           "Parameter ID","Parameter Entity Type",
                                                           "Boolean:Is_List", "Boolean:Redact"]]
            
            intent_dict["target_lang"] = target_lang
            intent_dict["intent_name"] = intent_name
            intent_dict["intent_display_name"] = intent_names_dict.get(intent_name)
            assembled_intent = assemble_intent(intent_dict)
            lang_code = training_phrases_df.loc[1,target_lang]
            try:
                update_intent(assembled_intent, lang_code)
            except Exception as e:
                print(f'Exception at update_intent(assembled_intent,lang_code).\nException e: {e}\n')
                print(f'assembled_intent content:\n{assembled_intent}\n')
                raise Exception(e)
    print(f"\nCOMPLETED:\tupdate_all_intents() in {timer()-start_time}s\n")

########################################
def update_intents_by_lang(target_lang):
    start_time = timer()
    print(f"START:\t\tupdate_intents_by_lang({target_lang})\n")
    
    training_phrases_df = read_sheet(SheetsName.Training_Phrases.name)
    parameters_df = read_sheet(SheetsName.Parameters.name)

    intent_names = training_phrases_df.loc[3:,"Intent Name"].unique().tolist()
    intent_names_dict = {}
    for intent_name in intent_names:
        intent_names_dict[intent_name] = training_phrases_df[training_phrases_df["Intent Name"]==intent_name]["Intent Display Name"].values[0]

    for intent_name in intent_names_dict.keys():
        print(f'\nintent_name:[{intent_name}]\nintent_display_name:[{intent_names_dict.get(intent_name)}], target_lang:[{target_lang}]')
        intent_dict = {}
        intent_dict["training_phrases"] = training_phrases_df.loc[training_phrases_df["Intent Name"]==intent_name,
                                                                  ["Intent Name", "Intent Display Name", target_lang]]

        intent_dict["parameters"] = parameters_df.loc[parameters_df["Intent Name"]==intent_name,
                                                      ["Intent Name","Intent Display Name",
                                                       "Parameter ID","Parameter Entity Type",
                                                       "Boolean:Is_List", "Boolean:Redact"]]

        intent_dict["target_lang"] = target_lang
        intent_dict["intent_name"] = intent_name
        intent_dict["intent_display_name"] = intent_names_dict.get(intent_name)
        assembled_intent = assemble_intent(intent_dict)
        lang_code = training_phrases_df.loc[1,target_lang]
        try:
            update_intent(assembled_intent, lang_code)
        except Exception as e:
            print(f'Exception at update_intent(assembled_intent,lang_code).\nException e: {e}\n')
            print(f'assembled_intent content:\n{assembled_intent}\n')
            raise Exception(e)
    print(f"\nCOMPLETED:\tupdate_intents_by_lang({target_lang}) in {timer()-start_time}s\n")


## Entities

In [ ]:
##########################################################################
def update_entity(entity_type, lang_code):
    print(f"START: update_entity(entity_type:[{entity_type.display_name}],lang_code:[{lang_code}])")

    entity_client = cx_services.entity_types.EntityTypesClient(credentials=get_cx_credentials(), 
                                                               client_options=get_cx_client_options())
    entity_request = cx_types.UpdateEntityTypeRequest()
    entity_request.entity_type = entity_type
    entity_request.language_code = lang_code
    update_mask = field_mask_pb2.FieldMask(paths=['entities'])
    entity_request.update_mask = update_mask
    
    delay_next_cx_request()
    start_time = timer()
    response = entity_client.update_entity_type(request=entity_request)
    #print(response)
    print(f"COMPLETED: update_entity(entity_type:[{entity_type.display_name}],lang_code:[{lang_code}]) in {timer()-start_time}s\n")

############################################
def update_entities(entities_df, lang_code):
    entities = []

    columns = entities_df.columns
    target_lang = columns[len(columns)-1]
    print(f"Language: [{target_lang}]")
    entity_type_names = entities_df['Entity Type Name'].unique()

    for entity_type_name in entity_type_names:
        entity_name_filter = entities_df["Entity Type Name"] == entity_type_name
        entity_type = cx_types.EntityType()
        entity_type.name = entity_type_name
        entity_type.display_name = entities_df.loc[entity_name_filter,"Entity Type Display Name"].values[0] 

        int_kind = int( entities_df.loc[entity_name_filter,"Entity Type Kind"].values[0] )
        entity_type.kind = cx_types.EntityType.Kind(int_kind)
        
        entity_type_values = entities_df.loc[entity_name_filter,'Entities Value'].unique()
        for entity_type_value in entity_type_values:
            entity = cx_types.EntityType.Entity()
            entity.value = entity_type_value
            synonyms_list = []
            if int_kind == 1:
                synonyms_list.append(entity.value)

            synonyms_df = entities_df[ (entity_name_filter) & (entities_df["Entities Value"] == entity_type_value) ]
            for synonym in synonyms_df[target_lang]:
                synonyms_list.append(synonym)
            entity.synonyms = synonyms_list
            entity_type.entities.append(entity)
            
        try:
            update_entity(entity_type,lang_code)
        except Exception as e:
            print(f"update_entities(entities_df, lang_code={lang_code}):")
            print(f"Exception at update_entity(entity_type=[see below], lang_code={lang_code}).")
            print(f"entity_type:\n{entity_type}\n")
            print(f"Exception e:\n{e}\n\nContinuing with next iteration")
    

#########################
def update_all_entities():
    start_time = timer()
    print("START:\t\tupdate_all_entities()\n")
    
    df = read_sheet(SheetsName.Entities.name)
    print(f'update_all_entities(): num rows in {SheetsName.Entities.name} sheet: {len(df)}')
    target_langs = df.columns.tolist()
    del target_langs[0:5]

    for target_lang in target_langs:
        lang_code = df.loc[1,target_lang]
        print(f"lang_code:{lang_code}")
        entities_frame = df.loc[3:len(df),[df.columns[0],df.columns[1],df.columns[2],df.columns[3],target_lang]]
        #print(entities_frame)
        update_entities(entities_frame, lang_code)
    
    print(f"\nCOMPLETED:\tupdate_all_entities() in {timer()-start_time}s\n")

#################################        
def update_entities_by_lang(target_lang):
    start_time = timer()
    print(f"START:\t\tupdate_entities_by_lang({target_lang})\n")
    
    df = read_sheet(SheetsName.Entities.name)
    print(f'update_entities_by_lang(target_lang={target_lang}): num rows in {SheetsName.Entities.name} sheet: {len(df)}')

    lang_code = df.loc[1,target_lang]
    print(f"lang_code:{lang_code}")
    entities_frame = df.loc[3:len(df),[df.columns[0],df.columns[1],df.columns[2],df.columns[3],target_lang]]
    #print(entities_frame)
    update_entities(entities_frame, lang_code)
    
    print(f"\nCOMPLETED:\tupdate_entities_by_lang({target_lang}) in {timer()-start_time}s\n")


## Flows

- name
- display_name
- transition_routes (trigger_fulfillment with messages:text/s,output_audio_text)
- event_handlers (trigger_fulfillment with text/s)

In [ ]:
##########################################################################
def update_flow(flow, lang_code):
    # Update transition_routes and event_handlers only
    # transition_route_groups via a separate process
    # 1. Query the Flow (source) from CX based on flow.name
    # 2. Merge the changes in transition_routes & event_handlers
    # 3. Update the Flow
    print(f"START: update_flow(flow:[{flow.display_name}],lang_code:[{lang_code}])")
    #print(flow)
    flows_client = cx_services.flows.FlowsClient(credentials=get_cx_credentials(), 
                                                 client_options=get_cx_client_options())    
    # 1. Get source Flow from CX
    src_Flow = get_flow(flow.name, lang_code) 
    
    #print(src_Flow)
    
    # 2a. transition_routes
    src_transition_routes = src_Flow.transition_routes
    trg_transition_routes = flow.transition_routes
    for trg_t_route in trg_transition_routes:
        if trg_t_route.intent.startswith('projects'):
            for src_t_route in src_transition_routes:
                if trg_t_route.intent == src_t_route.intent:
                    src_t_route.trigger_fulfillment.messages = merge_messages(src_t_route.trigger_fulfillment.messages,
                                                                              trg_t_route.trigger_fulfillment.messages)
        else:
            for src_t_route in src_transition_routes:
                if trg_t_route.condition == src_t_route.condition:
                    src_t_route.trigger_fulfillment.messages = merge_messages(src_t_route.trigger_fulfillment.messages,
                                                                              trg_t_route.trigger_fulfillment.messages)
    
    # 2b. event_handlers
    src_event_handlers = src_Flow.event_handlers
    trg_event_handlers = flow.event_handlers
    for trg_ev_handler in trg_event_handlers:
        for src_ev_handler in src_event_handlers:
            if trg_ev_handler.event == src_ev_handler.event:
                src_ev_handler.trigger_fulfillment.messages = merge_messages(src_ev_handler.trigger_fulfillment.messages,
                                                                             trg_ev_handler.trigger_fulfillment.messages)
    
    flow.transition_routes = src_transition_routes 
    flow.event_handlers = src_event_handlers
    #print(flow)
    
    # 3. Update Flow to CX
    flow_request = cx_types.UpdateFlowRequest()
    flow_request.flow = flow
    flow_request.language_code = lang_code
    update_mask = field_mask_pb2.FieldMask(paths=['transition_routes','event_handlers'])
    flow_request.update_mask = update_mask
    
    delay_next_cx_request()
    start_time = timer()
    
    response = flows_client.update_flow(request=flow_request)
    
    print(f"COMPLETED: update_flow(flow:[{flow.display_name}],lang_code:[{lang_code}]) in {timer()-start_time}s\n")
    #print(response)


########################################
def update_flows(flows_df, lang_code):

    columns = flows_df.columns
    print(f"Language: [{columns[len(columns)-1]}]")
    
    flow_names = flows_df['Flow Name'].unique()
    #print(flow_names)
    for flow_name in flow_names:
        flow = cx_types.Flow()
        flow.name = flow_name
        flow_df = flows_df[flows_df[columns[0]] == flow_name]
        flow.display_name = flow_df.iloc[0,1]
        # transition_routes, event_handlers and transition_route_groups
        
        # Create transition_routes
        t_routes_df = flow_df[flow_df['Flow Components'] == 'transition_routes']
        t_routes_IDs = t_routes_df['Flow Component ID'].unique()
        transition_routes = []
        for t_routes_id in t_routes_IDs:
            t_routes_id_df = t_routes_df[t_routes_df['Flow Component ID'] == t_routes_id]
            #print(t_routes_intent_df)
            text_list = []
            ssml_list = []
            for text in t_routes_id_df[t_routes_id_df.columns[len(t_routes_id_df.columns)-1]]:
                #print(text)
                if isSSML(text): #ssml
                    ssml_list.append(text)
                else: #text
                    text_list.append(text)
            
            messages = []
            if text_list != []:
                text = cx_types.ResponseMessage.Text()
                text.text = text_list
                #print(text)
                message = cx_types.ResponseMessage()
                message.text = text
                messages.append(message)
            if ssml_list != []:
                output_audio_texts = []
                for ssml in ssml_list:
                    output_audio_text = cx_types.ResponseMessage.OutputAudioText()
                    output_audio_text.ssml = ssml
                    output_audio_texts.append(output_audio_text)
                #print(output_audio_texts)
                for oat in output_audio_texts:
                    message = cx_types.ResponseMessage()
                    message.output_audio_text = oat
                    messages.append(message)
            #print(messages)
            fulfillment = cx_types.Fulfillment()
            fulfillment.messages = messages
            transition_route = cx_types.TransitionRoute()
            
            if t_routes_id.startswith('projects'):
                transition_route.intent = t_routes_id
            else:
                transition_route.condition = t_routes_id
            
            transition_route.trigger_fulfillment = fulfillment
            transition_routes.append(transition_route)
        
        flow.transition_routes = transition_routes
        
        #Create event_handlers
        ev_handlers_df = flow_df[flow_df['Flow Components'] == 'event_handlers']
        ev_handlers_IDs = ev_handlers_df['Flow Component ID'].unique()
        event_handlers = []
        for ev_handlers_id in ev_handlers_IDs:
            ev_handlers_id_df = ev_handlers_df[ev_handlers_df['Flow Component ID'] == ev_handlers_id]
            #print(ev_handlers_id_df)
            text_list = []
            ssml_list = []
            for text in ev_handlers_id_df[ev_handlers_id_df.columns[len(ev_handlers_id_df.columns)-1]]:
                #print(text)
                if isSSML(text): #ssml
                    ssml_list.append(text)
                else: #text
                    text_list.append(text)
            
            messages = []
            if text_list != []:
                text = cx_types.ResponseMessage.Text()
                text.text = text_list
                #print(text)
                message = cx_types.ResponseMessage()
                message.text = text
                messages.append(message)
            if ssml_list != []:
                output_audio_texts = []
                for ssml in ssml_list:
                    output_audio_text = cx_types.ResponseMessage.OutputAudioText()
                    output_audio_text.ssml = ssml
                    output_audio_texts.append(output_audio_text)
                #print(output_audio_texts)
                for oat in output_audio_texts:
                    message = cx_types.ResponseMessage()
                    message.output_audio_text = oat
                    messages.append(message)
            #print(messages)
            fulfillment = cx_types.Fulfillment()
            fulfillment.messages = messages
            event_handler = cx_types.EventHandler()
            event_handler.event = ev_handlers_id
            event_handler.trigger_fulfillment = fulfillment
            event_handlers.append(event_handler)
        
        flow.event_handlers = event_handlers
        
        #print(flow)
        try:
            update_flow(flow,lang_code)
        except Exception as e:
            print(f"update_flows(flows_df, lang_code={lang_code}):")
            print(f"Exception at update_flow(flow=[see below], lang_code={lang_code}).")
            print(f"flow:\n{flow}\n")
            print(f"Exception e:\n{e}\n\nContinuing with next iteration")
            #raise Exception(e)
        


#########################
def update_all_flows():
    start_time = timer()
    print("START:\t\tupdate_all_flows()\n")
    
    df = read_sheet(SheetsName.Flows.name)
    print(f'update_all_flows(): num rows in {SheetsName.Flows.name} sheet: {len(df)}')
    target_langs = df.columns.tolist()
    del target_langs[0:5]

    for target_lang in target_langs:
        lang_code = df.loc[1,target_lang]
        print(f"lang_code:{lang_code}")
        flows_frame = df.loc[3:len(df),[df.columns[0],df.columns[1],df.columns[2],df.columns[3],target_lang]]
        #print(flows_frame)
        update_flows(flows_frame, lang_code)
    
    print(f"\nCOMPLETED:\tupdate_all_flows() in {timer()-start_time}s\n")

#################################        

def update_flows_by_lang(target_lang):
    start_time = timer()
    print(f"START:\t\tupdate_flows_by_lang({target_lang})\n")
    
    df = read_sheet(SheetsName.Flows.name)
    print(f'update_all_flows(): num rows in {SheetsName.Flows.name} sheet: {len(df)}')

    lang_code = df.loc[1,target_lang]
    print(f"lang_code:{lang_code}")
    flows_frame = df.loc[3:len(df),[df.columns[0],df.columns[1],df.columns[2],df.columns[3],target_lang]]
    #print(flows_frame)
    update_flows(flows_frame, lang_code)
    
    print(f"\nCOMPLETED:\tupdate_flows_by_lang({target_lang}) in {timer()-start_time}s\n")
        

## Pages

- name
- display_name
- entry_fulfillment
- form
- transition_routes (trigger_fulfillment with messages:text/s,output_audio_text)
- event_handlers (trigger_fulfillment with text/s)

In [ ]:
#################################
def update_page(page, lang_code):
    # 1. Query the Page (source) from CX based on page.name
    # 2. Merge the changes in entry_fulfillment, Form/Parameters, transition_routes & event_handlers
    # 3. Update the Page
    print(f"START: update_page(page:[{page.display_name}],lang_code:[{lang_code}])")
    #print(page)
    pages_client = cx_services.pages.PagesClient(credentials=get_cx_credentials(), 
                                                 client_options=get_cx_client_options())    
    # 1. Get source Page from CX
    src = get_page(page.name, lang_code) 
    #print(src)
    
    # 2a. entry_fulfillment
    src.entry_fulfillment.messages = merge_messages(src.entry_fulfillment.messages, 
                                                    page.entry_fulfillment.messages)
    
    # 2b. form
    if page.form.parameters != []:
        i = 0
        while i < len(page.form.parameters):
            src.form.parameters[i].fill_behavior.initial_prompt_fulfillment.messages = merge_messages(src.form.parameters[i].fill_behavior.initial_prompt_fulfillment.messages, 
                                                                                                      page.form.parameters[i].fill_behavior.initial_prompt_fulfillment.messages)
            if src.form.parameters[i].fill_behavior.reprompt_event_handlers == []:
                src.form.parameters[i].fill_behavior.reprompt_event_handlers = page.form.parameters[i].fill_behavior.reprompt_event_handlers
            else:
                j = 0
                while j < len(src.form.parameters[i].fill_behavior.reprompt_event_handlers):
                    if src.form.parameters[i].fill_behavior.reprompt_event_handlers[j].trigger_fulfillment == []:
                        src.form.parameters[i].fill_behavior.reprompt_event_handlers[j].trigger_fulfillment = page.form.parameters[i].fill_behavior.reprompt_event_handlers[j].trigger_fulfillment
                    else:
                        src.form.parameters[i].fill_behavior.reprompt_event_handlers[j].trigger_fulfillment.messages = merge_messages(src.form.parameters[i].fill_behavior.reprompt_event_handlers[j].trigger_fulfillment.messages,
                                                                                                                                     page.form.parameters[i].fill_behavior.reprompt_event_handlers[j].trigger_fulfillment.messages)
                    j += 1
            i += 1
    
    # 2c. transition_routes
    if page.transition_routes != []:
        for tgt_t_route in page.transition_routes:
            if tgt_t_route.intent.startswith('projects'):
                for src_t_route in src.transition_routes:
                    if tgt_t_route.intent == src_t_route.intent:
                        src_t_route.trigger_fulfillment.messages = merge_messages(src_t_route.trigger_fulfillment.messages, 
                                                                                  tgt_t_route.trigger_fulfillment.messages)
            else:
                for src_t_route in src.transition_routes:
                    if tgt_t_route.condition == src_t_route.condition:
                        src_t_route.trigger_fulfillment.messages = merge_messages(src_t_route.trigger_fulfillment.messages,
                                                                                  tgt_t_route.trigger_fulfillment.messages)
    
    # 2d. event_handlers
    if page.event_handlers != []:
        for tgt_ev_handler in page.event_handlers:
            for src_ev_handler in src.event_handlers:
                if tgt_ev_handler.event == src_ev_handler.event:
                    src_ev_handler.trigger_fulfillment.messages = merge_messages(src_ev_handler.trigger_fulfillment.messages,
                                                                                 tgt_ev_handler.trigger_fulfillment.messages)

    # 3. Update Page to CX
    page_request = cx_types.UpdatePageRequest()
    page_request.page = src
    page_request.language_code = lang_code
    update_mask = field_mask_pb2.FieldMask(paths=['entry_fulfillment', 'form', 'transition_routes','event_handlers'])
    page_request.update_mask = update_mask
    
    delay_next_cx_request()
    start_time = timer()
    
    response = pages_client.update_page(request=page_request)
    
    print(f"COMPLETED: update_page(page:[{page.display_name}],lang_code:[{lang_code}]) in {timer()-start_time}s\n")
    #print(response)


In [ ]:
######################################
def update_pages(pages_df, lang_code):

    columns = pages_df.columns
    print(f"Language: [{columns[len(columns)-1]}]")
    
    page_names = pages_df['Page Name'].unique()
    #print(pages_names)
    for page_name in page_names:
        page = cx_types.Page()
        page.name = page_name
        page_df = pages_df[pages_df['Page Name'] == page_name]
        page.display_name = page_df.iloc[0,2]
        
        # entry_fulfillment, form/parameters, transition_routes, event_handlers
        
        # Create entry_fulfillment
        ef_df = page_df[page_df['Page Components'] == 'entry_fulfillment']
        #print(f'<entry fulfillment df>\n{ef_df}\n </entry fulfillment df>\n')
        if not ef_df.empty:
            fulfillment = cx_types.Fulfillment()
            fulfillment.messages = generate_cx_messages(ef_df)
            page.entry_fulfillment = fulfillment
        
        # Create form/parameters
        params_df = page_df[page_df['Page Components'] == 'parameter']
        #print(f'\n<params df>\n{params_df}\n</params df>\n')
        params    = params_df['Page Component ID'].unique()
        #print(f'params: {params}')
        form = cx_types.Form()
        parameters = []
        for param in params:
            param_df = params_df[params_df['Page Component ID'] == param]
            fillBehaviors = param_df['Page Component ID-2'].unique()
            
            parameter = cx_types.Form.Parameter()
            parameter.display_name = param
            fill_behavior = cx_types.Form.Parameter.FillBehavior()
            event_handlers = []
            
            for fillBehavior in fillBehaviors:
                fillbehavior_df = param_df[param_df['Page Component ID-2'] == fillBehavior]
                fulfillment = cx_types.Fulfillment()
                if fillBehavior == 'initial_prompt_fulfillment':
                    fulfillment.messages = generate_cx_messages(fillbehavior_df)
                    fill_behavior.initial_prompt_fulfillment = fulfillment
                else: #reprompt event handlers
                    fulfillment.messages = generate_cx_messages(fillbehavior_df)
                    event_handler = cx_types.EventHandler()
                    event_handler.event = fillBehavior
                    event_handler.trigger_fulfillment = fulfillment
                    event_handlers.append(event_handler)
            
            if len(event_handlers) > 0:
                fill_behavior.reprompt_event_handlers = event_handlers
            parameter.fill_behavior = fill_behavior
            parameters.append(parameter)
            
        if parameters != []:
            form.parameters = parameters
            #print(f'form:{form}')
            page.form = form
        
        # Create transition_routes
        t_routes_df = page_df[page_df['Page Components'] == 'transition_routes']
        t_routes_IDs = t_routes_df['Page Component ID'].unique()
        #print(f'\n<transition_routes df>\n{t_routes_df}\n</transition_routes df>\n')
        #print(f't_routes_IDs:{t_routes_IDs}')
        transition_routes = []
        for t_routes_id in t_routes_IDs:
            t_routes_id_df = t_routes_df[t_routes_df['Page Component ID'] == t_routes_id]
            fulfillment = cx_types.Fulfillment()
            fulfillment.messages = generate_cx_messages(t_routes_id_df)
            
            transition_route = cx_types.TransitionRoute()
            if t_routes_id.startswith('projects'):
                transition_route.intent = t_routes_id
            else:
                transition_route.condition = t_routes_id
            
            transition_route.trigger_fulfillment = fulfillment
            transition_routes.append(transition_route)
        
        if transition_routes != []:
            page.transition_routes = transition_routes
        
        #Create event_handlers
        ev_handlers_df = page_df[page_df['Page Components'] == 'event_handlers']
        #print(f'\n<event handlers df>\n{ev_handlers_df}\n</event handlers df>\n')
        ev_handlers_IDs = ev_handlers_df['Page Component ID'].unique()
        event_handlers = []
        for ev_handlers_id in ev_handlers_IDs:
            ev_handlers_id_df = ev_handlers_df[ev_handlers_df['Page Component ID'] == ev_handlers_id]
            fulfillment = cx_types.Fulfillment()
            fulfillment.messages = generate_cx_messages(ev_handlers_id_df)
            event_handler = cx_types.EventHandler()
            event_handler.event = ev_handlers_id
            event_handler.trigger_fulfillment = fulfillment
            event_handlers.append(event_handler)
        
        if event_handlers != []:
            page.event_handlers = event_handlers
        
        #print(page)
        try:
            update_page(page,lang_code)
        except Exception as e:
            print(f"update_pages(pages_df, lang_code={lang_code}):")
            print(f"Exception at update_page(page=[see below], lang_code={lang_code}).")
            print(f"page:\n{page}\n")
            print(f"Exception e:\n{e}\n\nContinuing with next iteration")
            

In [ ]:
#######################
def update_all_pages():
    start_time = timer()
    print("START:\t\tupdate_all_pages()\n")
    
    df = read_sheet(SheetsName.Pages.name)
    print(f'update_all_pages(): num rows in {SheetsName.Pages.name} sheet: {len(df)}')
    target_langs = df.columns.tolist()
    del target_langs[0:7]
    
    for target_lang in target_langs:
        lang_code = df.loc[1,target_lang]
        print(f"lang_code:{lang_code}")
        pages_frame = df.loc[3:len(df),[df.columns[0],df.columns[1],df.columns[2],df.columns[3],
                                        df.columns[4],df.columns[5],target_lang]]
        #print(pages_frame)
        update_pages(pages_frame, lang_code)
    
    print(f"\nCOMPLETED:\tupdate_all_pages() in {timer()-start_time}s\n")

######################################
def update_pages_by_lang(target_lang):
    start_time = timer()
    print(f"START:\t\tupdate_pages_by_lang({target_lang})\n")
    
    df = read_sheet(SheetsName.Pages.name)
    print(f'update_all_pages(): num rows in {SheetsName.Pages.name} sheet: {len(df)}')

    lang_code = df.loc[1,target_lang]
    print(f"lang_code:{lang_code}")
    pages_frame = df.loc[3:len(df),[df.columns[0],df.columns[1],df.columns[2],df.columns[3],
                                    df.columns[4],df.columns[5],target_lang]]
    #print(pages_frame)
    update_pages(pages_frame, lang_code)
    
    print(f"\nCOMPLETED:\tupdate_pages_by_lang({target_lang}) in {timer()-start_time}s\n")


## Route Groups

In [ ]:
###############################################
def update_route_group(route_group, lang_code):
    # 1. Query the Route Group (source) from CX based on route_group.name
    # 2. Merge the changes in transition_routes
    # 3. Update the Route Group
    print(f"START: update_route_group(route_group:[{route_group.display_name}],lang_code:[{lang_code}])")
    #print(route_group)
    trgs_client = cx_services.transition_route_groups.TransitionRouteGroupsClient(credentials=get_cx_credentials(), 
                                                                                  client_options=get_cx_client_options())    
    # 1. Get source Route Group from CX
    src = get_route_group(route_group.name, lang_code) 
    #print(src)
    
    # 2. transition_routes
    if route_group.transition_routes != []:
        for tgt_t_route in route_group.transition_routes:
            if tgt_t_route.intent.startswith('projects'):
                for src_t_route in src.transition_routes:
                    if tgt_t_route.intent == src_t_route.intent:
                        src_t_route.trigger_fulfillment.messages = merge_messages(src_t_route.trigger_fulfillment.messages, 
                                                                                  tgt_t_route.trigger_fulfillment.messages)
            else:
                for src_t_route in src.transition_routes:
                    if tgt_t_route.condition == src_t_route.condition:
                        src_t_route.trigger_fulfillment.messages = merge_messages(src_t_route.trigger_fulfillment.messages,
                                                                                  tgt_t_route.trigger_fulfillment.messages)

    # 3. Update Route Group to CX
    route_group_request = cx_types.UpdateTransitionRouteGroupRequest()
    route_group_request.transition_route_group = src
    route_group_request.language_code = lang_code
    update_mask = field_mask_pb2.FieldMask(paths=['transition_routes'])
    route_group_request.update_mask = update_mask

    delay_next_cx_request()
    start_time = timer()
    
    response = trgs_client.update_transition_route_group(request=route_group_request)
    
    print(f"COMPLETED: update_route_group(route_group:[{route_group.display_name}],lang_code:[{lang_code}]) in {timer()-start_time}s\n")
    #print(response)

In [ ]:
###########################################
def update_route_groups(rgs_df, lang_code):

    columns = rgs_df.columns
    print(f"Language: [{columns[len(columns)-1]}]")
    
    rg_names = rgs_df['Route Group Name'].unique()
    #print(pages_names)
    for rg_name in rg_names:
        route_group = cx_types.TransitionRouteGroup()
        route_group.name = rg_name
        rg_df = rgs_df[rgs_df['Route Group Name'] == rg_name]
        route_group.display_name = rg_df.iloc[0,2]
        
        # transition_routes
        
        # Create transition_routes
        t_routes_IDs = rg_df['Route Group Component ID'].unique()
        #print(f't_routes_IDs:{t_routes_IDs}')
        transition_routes = []
        for t_routes_id in t_routes_IDs:
            t_routes_id_df = rg_df[rg_df['Route Group Component ID'] == t_routes_id]
            fulfillment = cx_types.Fulfillment()
            fulfillment.messages = generate_cx_messages(t_routes_id_df)
            
            transition_route = cx_types.TransitionRoute()
            if t_routes_id.startswith('projects'):
                transition_route.intent = t_routes_id
            else:
                transition_route.condition = t_routes_id
            
            transition_route.trigger_fulfillment = fulfillment
            transition_routes.append(transition_route)
        
        if transition_routes != []:
            route_group.transition_routes = transition_routes
        
        #print(route_group)
        try:
            update_route_group(route_group,lang_code)
        except Exception as e:
            print(f"update_route_groups(rgs_df, lang_code={lang_code}):")
            print(f"Exception at update_route_group(route_group=[see below], lang_code={lang_code}).")
            print(f"route_group:\n{route_group}\n")
            print(f"Exception e:\n{e}\n\nContinuing with next iteration")


In [ ]:
##############################
def update_all_route_groups():
    start_time = timer()
    print("START:\t\tupdate_all_route_groups()\n")
    
    df = read_sheet(SheetsName.Route_Groups.name)
    print(f'update_all_route_groups(): num rows in {SheetsName.Route_Groups.name} sheet: {len(df)}')
    target_langs = df.columns.tolist()
    del target_langs[0:5]

    for target_lang in target_langs:
        lang_code = df.loc[1,target_lang]
        print(f"lang_code:{lang_code}")
        rgs_frame = df.loc[3:len(df),[df.columns[0],df.columns[1],df.columns[2],df.columns[3],target_lang]]
        #print(rgs_frame)
        update_route_groups(rgs_frame, lang_code)
    
    print(f"\nCOMPLETED:\tupdate_all_route_groups() in {timer()-start_time}s\n")
    
#############################################
def update_route_groups_by_lang(target_lang):
    start_time = timer()
    print(f"START:\t\tupdate_route_groups_by_lang({target_lang})\n")
    
    df = read_sheet(SheetsName.Route_Groups.name)
    print(f'update_route_groups_by_lang(): num rows in {SheetsName.Route_Groups.name} sheet: {len(df)}')

    lang_code = df.loc[1,target_lang]
    print(f"lang_code:{lang_code}")
    rgs_frame = df.loc[3:len(df),[df.columns[0],df.columns[1],df.columns[2],df.columns[3],target_lang]]
    #print(rgs_frame)
    update_route_groups(rgs_frame, lang_code)
    
    print(f"\nCOMPLETED:\tupdate_route_groups_by_lang({target_lang}) in {timer()-start_time}s\n")


# END

In [ ]:
print('Agent Notebook: RAN successfully to desired point')